# Datos
Si aún no cuentas con los datos, puedes descargarlos con el siguiente código.
Los datos deben estar en la carpeta `data`.

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("paultimothymooney/stock-market-data")

print("Path to dataset files:", path)

Path to dataset files: /home/antonio/.cache/kagglehub/datasets/paultimothymooney/stock-market-data/versions/74


Coloca los datos en la carpeta `data` y asegúrate de que la estructura de carpetas sea la siguiente:
```terminal
data
└── stock_market_data
    ├── forbes2000
    │   ├── csv
    │   └── json
    ├── nasdaq
    │   ├── csv
    │   └── json
    ├── nyse
    │   ├── csv
    │   └── json
    └── sp500
        ├── csv
        └── json
```

